In [1]:
# keras version

import keras
keras.__version__


#convnet filters
from keras import layers
from keras import models

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# import mnist data

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28,))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28*28,))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [3]:
#karnet parameters
f = ['atan','atan','atan','atan'] #% try setting H=[60000,8000,300] and slope = [1,1,1,1]
finv = ['tan','tan','tan','tan']  #% try setting H=[60000,8000,300] and slope = [1,1,1,1]
h = [2000,1000,100]
slope=1
rseed=1
x = train_images
y = train_labels

In [4]:
x.shape

(60000, 784)

In [5]:
# make karnet model
import numpy as np
import math
bias = np.ones_like(x[:,0]).reshape((-1,1))
bias.shape

(60000, 1)

In [6]:
X = np.concatenate([bias,x],axis=1) # add bias to inputs
nrow,ncol = X.shape
nrow,ncol

(60000, 785)

In [7]:
lenh = len(h)
len(h)+1

4

In [8]:
x.shape, y.shape

((60000, 784), (60000, 10))

In [9]:
#initialize
np.random.seed(rseed)
w,v = [],[]
for m in np.arange(len(h)):
    w_row = h[len(h)-1-m]
    if m==0:
        w_col = y.shape[1]
    else:
        w_col = h[len(h)-m]
    
    w.append(np.random.rand(w_row,w_col))
    v.append(np.random.rand(1,w_col))
w.append(np.zeros((0,0)))
v.append(np.zeros((0,0)))
w.reverse()
v.reverse()

In [10]:
#sigmoid function
def sigmoid(x):
    return 1 / (1 +np.exp(-x))

In [11]:
W1 = slope * np.linalg.pinv(X).dot(np.arctan(
    (np.arctan(
        (np.arctan(
            (np.arctan(y)-slope*bias.dot(v[3]))
            .dot(slope*np.linalg.pinv(w[3]))
        )-slope*bias.dot(v[2]))
        .dot(slope*np.linalg.pinv(w[2]))
    )-slope*bias.dot(v[1]))
    .dot(slope*np.linalg.pinv(w[1]))
))

In [12]:
W2 = slope * np.linalg.pinv(np.concatenate([bias,np.tanh(slope*X.dot(W1))],axis=1)).dot(
    np.arctanh(
        (np.arctanh(
            (np.arctanh(y)-slope*bias.dot(v[3]))
            .dot(slope*np.linalg.pinv(w[3]))
        )-slope*bias.dot(v[2]))
        .dot(slope*np.linalg.pinv(w[2]))
    ))

C:\Users\herok\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in arctanh
  after removing the cwd from sys.path.
C:\Users\herok\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arctanh
  """
C:\Users\herok\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in arctanh
  import sys


In [13]:
W3 = slope * np.linalg.pinv(
    np.concatenate([bias,
                    np.concatenate([bias,np.tanh(slope*X.dot(W1))],axis=1).dot(W2)],axis=1)
).dot(
np.arctanh(
            (np.arctanh(y)-slope*bias.dot(v[3]))
            .dot(slope*np.linalg.pinv(w[3]))
        ))


LinAlgError: SVD did not converge

In [ ]:
W4 = slope * np.linalg.pinv(
    np.concatenate([bias,
                    np.concatenate([bias,
                                    np.concatenate([bias,np.tanh(slope*X.dot(W1))],axis=1).dot(W2)],axis=1).dot(W3)],axis=1)
).dot(np.arctanh(y))

In [ ]:
W1.shape,W2.shape,W3.shape,W4.shape,

In [ ]:
#netoutstr
'atan(1.000000*[bias,atan(1.000000*[bias,atan(1.000000*[bias,atan(1.000000*X*W{1})]*W{2})]*W{3})]*W{4})'

In [ ]:
netout = np.arctanh(slope*
                   np.concatenate([bias,
                                        np.arctanh(slope*
                                                  np.concatenate([bias,
                                                                       np.arctanh(slope*
                                                                                 np.concatenate([bias,
                                                                                                 np.arctanh(slope*X.dot(W1))
                                                                                                      ],axis=1).dot(W2))
                                                                 ],axis=1).dot(W3))
                                  ],axis=1).dot(W4))

In [ ]:
netout[0]

In [ ]:
x = test_images
bias = np.ones_like(x[:,0]).reshape((-1,1))
X = np.concatenate([bias,x],axis=1) # add bias to inputs